In [ ]:
%load_ext autoreload
%autoreload 2

import os
from os import listdir
from os.path import isfile, join

import numpy as np
import matplotlib.pyplot as plt
import madmom

import sys
sys.path.append('../src')
from preprocessing import get_dataset
from models import OLSPatchRegressor

na = np.newaxis

plt.rc('text', usetex=True)
plt.rc('font', family='serif')

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [ ]:
music_dir  = '../data/music_speech/music_wav/'
speech_dir = '../data/music_speech/speech_wav/'

max_samples = -1

X, Y = get_dataset(music_dir, speech_dir, num_samples=max_samples, hpool=0, wpool=0)
print('Train Set Shape')
print(X.shape, Y.shape)

Y = (Y + 1) / 2 

# Train CNN using KERAS

In [ ]:
batch_size = 8
num_classes = 2
epochs = 109

# --------------------------------------------
num_frequencies = X.shape[1]
num_timesteps   = X.shape[2]
num_channels    = X.shape[3]
input_shape = num_frequencies, num_timesteps, num_channels

# DEFINE MODEL
model = Sequential()
model.add(MaxPooling2D(pool_size=(1, 3), input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(num_frequencies, 3),
                 activation='relu'))

model.add(Conv2D(1, kernel_size=(1, 1), activation='sigmoid'))
model.add(Lambda(lambda x: K.mean(x, axis=[1,2])))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
t_ind = 96
XTrain = X[:t_ind]
YTrain = Y[:t_ind]
XTest  = X[t_ind:]
YTest  = Y[t_ind:]

print(XTrain.shape, YTrain.shape)
print(XTest.shape, YTest.shape)

In [ ]:
model.fit(XTrain, YTrain,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(XTest, YTest))
score = model.evaluate(XTest, YTest, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model_path = '../models/keras/'
os.makedirs(model_path, exist_ok=True)

model.save(os.path.join(model_path, 'cnn_on_spectros_{:2.2f}.h5'.format(score[1])))